Student: Duncan Ferguson <br>
Student Id: 871641260 <br>
Class: Comp 4431-1 <br>
Assignment: Assignment 3 <br>
Date: 11/20/2021 <br>

You are to apply the following techniques to mine this data set:
- Association Rules
- Decision tree Classification
- Naive Bayes Classification
- kmeans and dbscan clustering

Also, you are to use Principle Component Analysis to reduce the number of attributes
(aka features)

Please answer the following questions:

1. What can you deduce from the data set? (in otherwords, what attribute values are indicative of "success")
2. Which mining techniques yielded the best results for what? How do you define "best results"? Please
include tables and/or graphs to justify your statement

Note: While numeric data is desired for classification/clustering, you need strings that look like market basket
data (sets of item names) for Association Rules. Numeric data can be grouped into names like
"lowValue, midValue, midValue, mediumHighValue, highValue" etc. See the following code for possible
way of doing this (not necessarily the best):

What to turn in:
- a pdf file with the answers to these three questions
- Your code used to answer these questions


In [22]:
# Importing Standard Libraries
import pandas as pd
import numpy as np


# Sklearn Libraries
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.naive_bayes import *
from sklearn.cluster import KMeans, DBSCAN
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

# Graphing Libraries
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Setting up view options
pd.set_option('display.max_columns', None)
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)
pd.options.display.width = 0   #


In [14]:
# Importing the Data Set

df =  pd.read_csv("bank.csv")
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [15]:
# First Exploration of all the Variables that we are dealing with
for col in df:
    print(col, ":", df[col].unique())

age : [30 33 35 59 36 39 41 43 20 31 40 56 37 25 38 42 44 26 55 67 53 68 32 49
 78 23 52 34 61 45 48 57 54 63 51 29 50 27 60 28 21 58 22 46 24 77 75 47
 70 65 64 62 66 19 81 83 80 71 72 69 79 73 86 74 76 87 84]
job : ['unemployed' 'services' 'management' 'blue-collar' 'self-employed'
 'technician' 'entrepreneur' 'admin.' 'student' 'housemaid' 'retired'
 'unknown']
marital : ['married' 'single' 'divorced']
education : ['primary' 'secondary' 'tertiary' 'unknown']
default : ['no' 'yes']
balance : [ 1787  4789  1350 ...  -333 -3313  1137]
housing : ['no' 'yes']
loan : ['no' 'yes']
contact : ['cellular' 'unknown' 'telephone']
day : [19 11 16  3  5 23 14  6 17 20 13 30 29 27  7 18 12 21 26 22  2  4 15  8
 28  9  1 10 31 25 24]
month : ['oct' 'may' 'apr' 'jun' 'feb' 'aug' 'jan' 'jul' 'nov' 'sep' 'mar' 'dec']
duration : [  79  220  185  199  226  141  341  151   57  313  273  113  328  261
   89  189  239  114  250  148   96  140  109  125  169  182  247  119
  149   74  897   81   40  958  35

Changing all non-numeric into category data

In [ ]:
df['job'] = df['job'].astype('category')
df['job'] = df['job'].cat.codes
df['marital'] = df['marital'].astype('category')
df['marital'] = df['marital'].cat.codes
df['education'] = df['education'].astype('category')
df['education'] = df['education'].cat.codes
df['default'] = df['default'].astype('category')
df['default'] = df['default'].cat.codes
df['contact'] = df['contact'].astype('category')
df['contact'] = df['contact'].cat.codes
df['month'] = df['month'].astype('category')
df['month'] = df['month'].cat.codes
df['poutcome'] = df['poutcome'].astype('category')
df['poutcome'] = df['poutcome'].cat.codes
df['housing'] = df['housing'].astype('category')
df['housing'] = df['housing'].cat.codes
df['loan'] = df['loan'].astype('category')
df['loan'] = df['loan'].cat.codes
df['y'] = df['y'].astype('category')
df['y'] = df['y'].cat.codes
df = df.drop('pdays',axis=1)
df['balance'] = (df['balance'] - df['balance'].min()) / ( df['balance'].max() - df['balance'].min())
print("Post Conversion of Data into numeric, dropping pdays, and normalizing balance:")
df.head(100)

In [18]:
# Taking a look at all the new unique values
for col in df:
    print(col, ":", df[col].unique())

age : [30 33 35 59 36 39 41 43 20 31 40 56 37 25 38 42 44 26 55 67 53 68 32 49
 78 23 52 34 61 45 48 57 54 63 51 29 50 27 60 28 21 58 22 46 24 77 75 47
 70 65 64 62 66 19 81 83 80 71 72 69 79 73 86 74 76 87 84]
job : [10  7  4  1  6  9  2  0  8  3  5 11]
marital : [1 2 0]
education : [0 1 2 3]
default : [0 1]
balance : [0.06845546 0.10875022 0.06258976 ... 0.03999946 0.         0.05973074]
housing : [0 1]
loan : [0 1]
contact : [0 2 1]
day : [19 11 16  3  5 23 14  6 17 20 13 30 29 27  7 18 12 21 26 22  2  4 15  8
 28  9  1 10 31 25 24]
month : [10  8  0  6  3  1  4  5  9 11  7  2]
duration : [  79  220  185  199  226  141  341  151   57  313  273  113  328  261
   89  189  239  114  250  148   96  140  109  125  169  182  247  119
  149   74  897   81   40  958  354  150   97  132  765   16  609  106
  365  205   11  105   59  425  204  181 1018 1740   98  441  272  159
  295  314  579  554  323  227  134  223  155  130  630  164  268  380
  154  221   67  367   87  701  652   63  398 

Creating X and Y for machine learning Algorithms

In [20]:
Y = df['y']
df = df.drop('y',axis=1)
X = df

Partitioning data into training and testing sets

In [23]:
x_train, x_test, y_train, y_test  = train_test_split( X, Y,  test_size=0.20, random_state=1)
print('len(x_train), len(y_train) = ' + str(len(x_train)) + ',' + str(len(y_train)) )
print('len(x_test), len(y_test) = ' + str(len(x_test)) + ',' + str(len(y_test)) )

len(x_train), len(y_train) = 3616,3616
len(x_test), len(y_test) = 905,905


Running a decision Tree

In [24]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(x_train,y_train)
print(clf)
y_predicted = clf.predict(x_test)
print('\nconfusion_matrix from decision tree:')
print(confusion_matrix( y_test, y_predicted ) )
accuracy = accuracy_score(y_test, y_predicted)
recall = recall_score(y_test, y_predicted)
precision = precision_score(y_test, y_predicted)
print('accuracy = ' + str(accuracy))
print('recall = ' + str(recall))
print('precision = ' + str(precision))

DecisionTreeClassifier()

confusion_matrix from decision tree:
[[731  59]
 [ 70  45]]
accuracy = 0.8574585635359117
recall = 0.391304347826087
precision = 0.4326923076923077


Running GaussianNB

In [25]:
model = GaussianNB()
model.fit(x_train,y_train)
gausianNB_predicted = model.predict(x_test)
print('\nconfusion_matrix from Gaussian naive bayes:')
print(confusion_matrix( y_test, gausianNB_predicted ) )
accuracy = accuracy_score(y_test, gausianNB_predicted)
recall = recall_score(y_test, gausianNB_predicted )
precision = precision_score(y_test, gausianNB_predicted)
print('accuracy = ' + str(accuracy))
print('recall = ' + str(recall))
print('precision = ' + str(precision))





confusion_matrix from Gaussian naive bayes:
[[731  59]
 [ 65  50]]
accuracy = 0.8629834254143647
recall = 0.43478260869565216
precision = 0.45871559633027525
